In [1]:
import napari
import numpy as np
from pathlib import Path
from tifffile import imread

from napari_cellseg3d.dev_scripts import evaluate_labels as eval
from napari_cellseg3d.utils import resize
from napari_cellseg3d.code_models.model_instance_seg import binary_connected, binary_watershed, voronoi_otsu

In [2]:
viewer = napari.Viewer()

In [3]:
im_path = Path("C:/Users/Cyril/Desktop/test/instance_test")
prediction_path = str(im_path / "pred.tif")
gt_labels_path = str(im_path / "labels_relabel_unique.tif")

prediction = imread(prediction_path)
gt_labels = imread(gt_labels_path)

zoom = (1/5,1,1)
prediction_resized = resize(prediction, zoom)
gt_labels_resized = resize(gt_labels, zoom)


viewer.add_image(prediction_resized, name='pred', colormap='inferno')
viewer.add_labels(gt_labels_resized, name='gt')
print(prediction_resized.shape)
print(gt_labels_resized.shape)

In the future `np.bool` will be defined as the corresponding NumPy scalar.
In the future `np.bool` will be defined as the corresponding NumPy scalar.
In the future `np.bool` will be defined as the corresponding NumPy scalar.
In the future `np.bool` will be defined as the corresponding NumPy scalar.


(25, 64, 64)
(25, 64, 64)


In [4]:
from napari_cellseg3d.dev_scripts.correct_labels import relabel
# gt_corrected = relabel(prediction_path, gt_labels_path, go_fast=False)

In [5]:
eval.evaluate_model_performance(gt_labels_resized, gt_labels_resized)

100%|██████████████████████████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 2926.92it/s]

Neurons found:  124
Neurons fused:  0
Neurons not found:  0
Artefacts found:  0
Mean true positive ratio of the model:  1.0
Mean ratio of the neurons pixels correctly labelled:  1.0
Mean ratio of the neurons pixels correctly labelled for fused neurons:  nan
Mean true positive ratio of the model for fused neurons:  nan
Mean ratio of false pixel in artefacts:  nan


(124, 0, 0, 0, 1.0, 1.0, nan, nan, nan)

In [6]:
connected = binary_connected(prediction_resized)
viewer.add_labels(connected,name='connected')

<Labels layer 'connected' at 0x2368a056d30>

In [7]:
eval.evaluate_model_performance(gt_labels_resized, connected)

100%|████████████████████████████████████████████████████████████████████████████████| 95/95 [00:00<00:00, 1912.60it/s]

Neurons found:  45
Neurons fused:  38
Neurons not found:  41
Artefacts found:  8
Mean true positive ratio of the model:  0.8424215218790255
Mean ratio of the neurons pixels correctly labelled:  0.9295584641244191
Mean ratio of the neurons pixels correctly labelled for fused neurons:  0.9332428837640889
Mean true positive ratio of the model for fused neurons:  0.8300682812799907
Mean ratio of false pixel in artefacts:  1.0


(45,
 38,
 41,
 8,
 0.8424215218790255,
 0.9295584641244191,
 0.9332428837640889,
 0.8300682812799907,
 1.0)

In [8]:
watershed = binary_watershed(prediction_resized, thres_small=20, rem_seed_thres=5)
viewer.add_labels(watershed)
eval.evaluate_model_performance(gt_labels_resized, watershed)

100%|████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 2290.34it/s]

Neurons found:  47
Neurons fused:  37
Neurons not found:  40
Artefacts found:  0
Mean true positive ratio of the model:  0.8426909426266451
Mean ratio of the neurons pixels correctly labelled:  0.9355733839977192
Mean ratio of the neurons pixels correctly labelled for fused neurons:  0.9369165405470844
Mean true positive ratio of the model for fused neurons:  0.8198206611354032
Mean ratio of false pixel in artefacts:  nan


(47,
 37,
 40,
 0,
 0.8426909426266451,
 0.9355733839977192,
 0.9369165405470844,
 0.8198206611354032,
 nan)

In [9]:
voronoi = voronoi_otsu(prediction_resized, 1, outline_sigma=0.5)
viewer.add_labels(voronoi)
voronoi.shape

(25, 64, 64)

In [10]:
# np.unique(voronoi, return_counts=True)

In [11]:
# np.unique(gt_labels, return_counts=True)

In [ ]:
eval.evaluate_model_performance(gt_labels_resized, voronoi)

 28%|██████████████████████▍                                                          | 34/123 [07:33<22:45, 15.34s/it]

In [ ]:
# eval.evaluate_model_performance(gt_labels_resized, voronoi)